In [ ]:
%pip install --quiet openai python-dotenv pillow pytesseract

In [ ]:
# Setup imports and client
from dotenv import load_dotenv
import os, io, base64, json
from PIL import Image

try:
    from openai import OpenAI
except Exception:
    OpenAI = None

load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_KEY) if (OpenAI and OPENAI_KEY) else None

if client is None:
    print('OpenAI client not initialized; falling back to heuristics only')
else:
    print('OpenAI client initialized')

In [ ]:
# Utility: encode image to base64
def encode_image_to_base64(image_path: str) -> str:
    img = Image.open(image_path).convert('RGB')
    buf = io.BytesIO()
    img.save(buf, format='JPEG')
    return base64.b64encode(buf.getvalue()).decode('utf-8')

# Heuristic fallback
def heuristic_classify(image_path: str) -> dict:
    category = 'other'
    explanation = 'Fallback heuristics used.'
    conf = 0.5
    name = os.path.basename(image_path).lower()
    if any(k in name for k in ['receipt','invoice','bill']):
        return {'category': 'receipt', 'confidence': 0.6, 'explanation': 'Filename suggests a receipt.'}
    try:
        import pytesseract
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img).lower()
        if any(k in text for k in ['total','subtotal','receipt','amount']):
            return {'category': 'receipt', 'confidence': 0.85, 'explanation': 'OCR detected receipt-like text.'}
        if any(k in text for k in ['prescription','rx','clinic','hospital','patient']):
            return {'category': 'health', 'confidence': 0.75, 'explanation': 'OCR detected medical text.'}
    except Exception:
        pass
    return {'category': category, 'confidence': conf, 'explanation': explanation}

In [ ]:
# Core classify function using OpenAI if available
def classify_image(image_path: str, allowed_categories=None, model='gpt-4o') -> dict:
    if allowed_categories is None:
        allowed_categories = ['scenery', 'food', 'receipt', 'health', 'collectable', 'other']
    if client is None:
        return heuristic_classify(image_path)

    system_prompt = ('You are an expert image classification assistant. The allowed output categories are: ' + ', '.join(allowed_categories) + '.\n'
                    + 'Return exactly a JSON object with keys: category (one of allowed categories), confidence (0.0-1.0), and explanation (short text). Do not add other text.')
    user_prompt = 'Classify the attached image into one of the categories and keep the output as strict JSON.'

    b64 = encode_image_to_base64(image_path)
    image_content = { 'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{b64}'} }
    messages = [ {'role': 'system', 'content': system_prompt}, {'role': 'user', 'content': [{'type': 'text', 'text': user_prompt}, image_content]} ]

    try:
        resp = client.chat.completions.create(model=model, messages=messages, max_tokens=200)
        raw = resp.choices[0].message.content
        try:
            parsed = json.loads(raw)
            if 'category' in parsed and parsed['category'] in allowed_categories:
                return {'category': parsed['category'], 'confidence': parsed.get('confidence', 0.9), 'explanation': parsed.get('explanation','')}
        except Exception:
            raw_l = raw.lower()
            for c in allowed_categories:
                if c in raw_l:
                    return {'category': c, 'confidence': 0.65, 'explanation': 'Found category word in model response.'}
            return {'category': 'other', 'confidence': 0.3, 'explanation': 'No category detected in model response.'}
    except Exception as e:
        print('OpenAI error:', e)
        return heuristic_classify(image_path)

In [ ]:
# Example usage
example_image_path = r"C:\Users\Ron\temp\IMG_6094.JPG"
result = classify_image(example_image_path, model='gpt-4o')
print('Category:', result.get('category'))
print('Confidence:', result.get('confidence'))
print('Explanation:', result.get('explanation'))